# Cohere Health Machine Learning Takehome Assignment
The following notebook was generated by members of the Cohere Health Machine Learning team and should be used for the purpose of progressing the interview takehome assignment. The code contained within this notebook is merely an example of how candidates can get started and is optional to use. Feel free to reach out to your recruiting team if you have any questions.

## Load in Data
You must provide YOUR OWN PATH to the location of the sampleclinicalnotes.zip file in the `PATH_TO_ZIP` object.

In [1]:
from load_data import load_ann, load_txt
import pandas as pd
import text_utils
import df_utils

In [2]:
PATH_TO_ZIP = "data/"
DATA_PATH = f"{PATH_TO_ZIP}training_20180910/"
print(f"Full data path: {DATA_PATH}")

Full data path: data/training_20180910/


In [3]:
# read in txt files
txt_df = load_txt(DATA_PATH)

Time taken to read .txt files: 0.03503155708312988


In [4]:
# read in REASONS entities from .ann files
ent_df, rel_df = load_ann(DATA_PATH)

Time taken to read .ann files and extract all metadata: 0.1531391143798828


In [5]:
all_dfs = [txt_df, ent_df, rel_df]
txt_df, ent_df, rel_df = [df_utils.fix_slashes(df) for df in all_dfs]

In [6]:
rel_df = df_utils.clean_rel_df(rel_df)
ent_df = df_utils.clean_ent_df(ent_df)

In [7]:
txt_df.head()

,file_idx,text
0,training_20180910/100035,Admission Date: [**2115-2-22**] ...
1,training_20180910/100039,Admission Date: [**2174-4-18**] ...
2,training_20180910/100187,Admission Date: [**2107-1-17**] ...
3,training_20180910/100229,Admission Date: [**2114-12-24**] ...
4,training_20180910/100564,Admission Date: [**2144-1-20**] ...


In [8]:
ent_df.head()

,file_idx,entity_id,category,start_idx,end_idx,text
0,training_20180910/100035,T1,Reason,10179,10197,recurrent seizures
1,training_20180910/100035,T3,Drug,10227,10233,ativan
2,training_20180910/100035,T5,Route,10240,10242,IM
3,training_20180910/100035,T6,Drug,10455,10465,Topiramate
4,training_20180910/100035,T7,Strength,10466,10470,25mg


In [9]:
rel_df.head()

,file_idx,relationship_id,category,entity1,entity2
0,training_20180910/100035,R1,Reason-Drug,T1,T3
1,training_20180910/100035,R4,Route-Drug,T5,T3
2,training_20180910/100035,R5,Strength-Drug,T7,T6
3,training_20180910/100035,R6,Route-Drug,T8,T6
4,training_20180910/100035,R7,Frequency-Drug,T9,T6


In [10]:
rel_df.category.value_counts()

category
Strength-Drug     6702
Form-Drug         6654
Frequency-Drug    6310
Route-Drug        5538
Reason-Drug       5169
Dosage-Drug       4225
ADE-Drug          1107
Duration-Drug      643
Name: count, dtype: int64

In [11]:
ent_df['length_diff'] = (ent_df.end_idx - ent_df.start_idx) - ent_df.text.str.len()

In [12]:
ent_df[(ent_df.category == 'Reason') & (ent_df.length_diff != 0)]

,file_idx,entity_id,category,start_idx,end_idx,text,length_diff
271,training_20180910/100035,T352,Reason,9314,9324,grand mal seizure,-7
581,training_20180910/100039,T294,Reason,25084,25092,prevent infection,-9
585,training_20180910/100039,T298,Reason,25152,25162,uric acid levels were high,-16
957,training_20180910/100187,T355,Reason,7425,7436,multifocal pneumonia,-9
984,training_20180910/100187,T382,Reason,8535,8553,breaking up thick sputum,-6
...,...,...,...,...,...,...,...
50715,training_20180910/196798,T368,Reason,14901,14911,continued outbursts,-9
50741,training_20180910/196798,T400,Reason,22979,22991,transiently febrile,-7
50746,training_20180910/196798,T405,Reason,23409,23418,Bacillus species grew from [**6-19**] PICC blo...,-42
50751,training_20180910/196798,T412,Reason,25470,25476,yeast infection,-9


In [13]:
txt_df[txt_df['file_idx'] == 'training_20180910/198406'].text.values[0][12981:12985+11]

'DVT\nprophylaxis'

In [14]:
df_utils.split_txt_df(txt_df)

,file_idx,text,diagnosis,history,complaint
0,training_20180910/100035,Admission Date: [**2115-2-22**] ...,Anoxic Brain Injury s/p PEA arrest x2\nStatus ...,Mr. [**Known lastname 3234**] is a 36 year old...,"Post-cardiac arrest, asthma exacerbation"
1,training_20180910/100039,Admission Date: [**2174-4-18**] ...,Primary:\n-Abdominal Pain\n-Acute on chronic r...,38 yo F w/ h/o ALL in remission s/p cord trans...,Abdominal Pain
2,training_20180910/100187,Admission Date: [**2107-1-17**] ...,Primary:\n1) Pulmonary Embolism with history o...,64 yo woman w/ h/o recurrent PEs s/[**Initials...,SOB
3,training_20180910/100229,Admission Date: [**2114-12-24**] ...,Primary:\n1. Sepsis\n2. Shock liver\n3. Hepari...,44 yoM PMH ESRD secondary to Brights disease o...,"Hypotension with elevated lactate, code sepsis."
4,training_20180910/100564,Admission Date: [**2144-1-20**] ...,1. Deep Vein Thrombosis of subclavian vein\n2....,"43 yo male with hx of rectal CA, DMII, and his...",SVC thrombosis
...,...,...,...,...,...
298,training_20180910/195689,Admission Date: [**2105-7-14**] ...,Acute Renal Failure\nHyperkalemia,Pt is a 64 y/o African American gentleman who ...,"Weakness, slurred speech, increased frequency ..."
299,training_20180910/195784,Admission Date: [**2175-7-11**] ...,HCV/HCC\ndeaf\nblind,"The patient is a 65-year-\nold male, with a hi...",HCC/HCV
300,training_20180910/196798,Admission Date: [**2167-4-28**] ...,"congestive heart failure , acute on chronic r...",This is a 53 year-old woman with history of CA...,Transferred from OSH with hypoxic respiratory ...
301,training_20180910/197869,Admission Date: [**2207-8-16**] ...,Lymphedema with superimposed cellulitis and un...,"56yo male with a history of DM2, CAD s/p CABG,...",Severe lower extremity lymphedema with superim...


In [17]:
print(txt_df.text[100])

Admission Date:  [**2158-4-27**]              Discharge Date:   [**2158-5-8**]

Date of Birth:  [**2085-8-9**]             Sex:   M

Service: CARDIOTHORACIC

Allergies:
No Known Allergies / Adverse Drug Reactions

Attending:[**First Name3 (LF) 165**]
Chief Complaint:
Chest pain

Major Surgical or Invasive Procedure:
[**2158-5-3**] - Urgent off-pump coronary artery bypass graft x4, left
internal mammary artery to left anterior descending artery and
saphenous vein grafts to diagonal, obtuse marginal and posterior
descending arteries.


History of Present Illness:
Mr. [**Known lastname 26644**] is [**Initials (NamePattern4) **] [**Last Name (NamePattern4) 664**] 72 yo with known coronary disease,
s/p MI'[**41**] s/p PCI/stent, s/p PCI [**2155**] now with increasing
episodes of chest tightness and
shortness of breath with exertion with minimal walking or
bending over.  He denies any history of chest pain.  A stress
test in [**Month (only) 404**] revealed moderate ischemia in the inferior
p

In [16]:
ent_df[ent_df.category == 'Reason']

,file_idx,entity_id,category,start_idx,end_idx,text,length_diff
0,training_20180910/100035,T1,Reason,10179,10197,recurrent seizures,0
21,training_20180910/100035,T27,Reason,11442,11451,agitation,0
23,training_20180910/100035,T29,Reason,11577,11586,agitation,0
25,training_20180910/100035,T32,Reason,11694,11700,Asthma,0
36,training_20180910/100035,T43,Reason,11988,11996,wheezing,0
...,...,...,...,...,...,...,...
50854,training_20180910/197869,T101,Reason,9054,9073,atrial fibrillation,0
50875,training_20180910/198406,T3,Reason,11476,11488,Pain control,0
50890,training_20180910/198406,T18,Reason,12981,12985,DVT prophylaxis,-11
50937,training_20180910/198406,T65,Reason,2888,2907,Atrial fibrillation,0
